In [1]:
# ==========================================
# 1. IMPORTACIONES (CRÍTICO: Esto faltaba)
# ==========================================
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, avg, round

# ==========================================
# 2. CONFIGURACIÓN DE LA SESIÓN
# ==========================================
# Usamos el paquete oficial de MongoDB para Spark
spark = SparkSession.builder \
    .appName("AutoInsights_ETL") \
    .config("spark.executor.memory", "2g") \
    .config("spark.driver.memory", "2g") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .config("spark.mongodb.input.uri", "mongodb://mongodb:27017/autoinsights.raw_data") \
    .config("spark.mongodb.output.uri", "mongodb://mongodb:27017/autoinsights.agregados") \
    .getOrCreate()

print(f"Spark Inicializado: {spark.version}")

Spark Inicializado: 3.5.0


In [2]:
# ==========================================
# 3. CARGA DE DATOS
# ==========================================
file_path = "/home/jovyan/work/data/vehicles.csv"

# inferSchema=True es cómodo pero lento. Para producción se define esquema manual.
df = spark.read.csv(file_path, header=True, inferSchema=True)
df.cache() # Guardamos en RAM para no leer el CSV dos veces

print(f"Total inicial: {df.count()}")

Total inicial: 441802


In [3]:
# ==========================================
# 4. LIMPIEZA (TRANSFORMATION)
# ==========================================
df_clean = df.filter(
    (col("price") > 500) & 
    (col("price") < 1000000) & 
    (col("year") > 1990) & 
    (col("year") <= 2025) &
    col("manufacturer").isNotNull() &
    col("model").isNotNull()
)

print(f"Total post-limpieza: {df_clean.count()}")

Total post-limpieza: 354813


In [10]:
# ==========================================
# 5. AGREGACIÓN (TRANSFORMATION)
# ==========================================
# Calculamos KPIs agrupados
df_agg = df_clean.groupBy("manufacturer", "model", "year") \
    .agg(
        round(avg("price"), 2).alias("avg_price"),
        count("*").alias("count")
    ) \
    .orderBy("manufacturer", "model", "year")

print("Muestra de datos a guardar:")
df_agg.show(5)

Muestra de datos a guardar:
+------------+---------------+-----+---------+-----+
|manufacturer|          model| year|avg_price|count|
+------------+---------------+-----+---------+-----+
|        2015|           2016| 2014|   2013.0|  164|
|       acura|2002 rsx type s| 2002|   7900.0|    1|
|       acura|          3.0cl| 1997|   2250.0|    2|
|       acura|          3.0cl| 1999|    800.0|    1|
|       acura|  3.2 cl type s| 2003|   8389.4|    5|
+------------+---------------+-----+---------+-----+
only showing top 5 rows



In [12]:
# ==========================================
# 6. ESCRITURA EN MONGODB (LOAD)
# ==========================================
print("Iniciando escritura en MongoDB...")

try:
    df_agg.write.format("mongo") \
        .mode("overwrite") \
        .option("database", "autoinsights") \
        .option("collection", "precios_promedio") \
        .save()
    
    print("¡ÉXITO! Datos guardados en MongoDB correctamente.")
    
except Exception as e:
    print(f"❌ Error al escribir en Mongo: {str(e)}")
    print("Tip: Verifica que el contenedor 'mongodb' esté corriendo y el conector se haya descargado.")

Iniciando escritura en MongoDB...
¡ÉXITO! Datos guardados en MongoDB correctamente.


In [14]:
# Verificación usando PyMongo (Librería estándar de Python)
# !pip install pymongo
from pymongo import MongoClient
import pprint

# 1. Conectar al contenedor de Mongo
# Nota: Usamos "mongodb" porque es el nombre del servicio en docker-compose
client = MongoClient("mongodb://mongodb:27017/")

# 2. Seleccionar base de datos y colección
db = client["autoinsights"]
collection = db["precios_promedio"]

# 3. Contar documentos
total = collection.count_documents({})
print(f"📉 Total de documentos en Mongo: {total}")

# 4. Mostrar un ejemplo (tal como lo verá la API)
print("\n🔍 Ejemplo de documento guardado:")
ejemplo = collection.find_one({"manufacturer": "toyota"}) # Buscamos un Toyota como prueba
pprint.pprint(ejemplo)

📉 Total de documentos en Mongo: 48741

🔍 Ejemplo de documento guardado:
{'_id': ObjectId('69654a9d8c989415b890b8fe'),
 'avg_price': 38997.0,
 'count': 13,
 'manufacturer': 'toyota',
 'model': '"4runner 1-arizona owner*0-rust*new bilstein toytec lift*new '
          '33""yokohama m/t*new black rhino wheels* 3rd seat*nav*black out '
          'pkg*0-accidents"',
 'year': '2018'}
